### serveMockBox

* data coming from the box is used by the **ISI_Dashboard** for realtime visualization of variables that are important to the experimenter. 
* here we serve a Mock of the box over localhost, that we can manipulate inorder to test and change the function of the **ISI_Dashboard**

### MockBox usage description: 

1. a client connects to the MockBox server
2. the MockBox Server sends a message to the client asking for it's "ID_REQ"
    1. this happens once and only once
3. if the "ID_REQ" == 1, then the client subscribes to JSON messages indefinitely.

### run the server

* to initiate and run the server, run the designated cell

#### stimulation system architecture

* stim_req_packets are sent in, at whatever frequency (could be 1khz)
* each time a packet is recieved, it overwrites a dictionary in memory which requests 


#### there is not 1 message, but a set of messages types that a logger subscribes to. These are: 

stim_event_json 
stim_ack_json
stim_req_json
soh_json

ID_REQ

#### all of these messages have the following structure.

{
"client_ip": str
"client_id": int OR "N/A"
"msg_type": str
"msg": str - sometimes JSON structured
}

this facilitates handling of different behaviors for different message types by first reading the 'msg_type' parameter.


#### understanding how electrode arrays are indexed:

arrays are indexed by a single number, either 1-32 or 129 - 161
1-32 is the first set of electrodes,  129-161 is the second
electrode arrays are numbered from top to bottom, and from left to right
so bottom left corner is 1,one to the right is 9, etc. 
you can figure out if this is something you want ot modify, or just stick with it.


#### how the stim packets are organized

within the stim_event_json packet's msg we have:

{
elecCath: int OR list of ints,
elecAno: int OR list of ints,
amp: int,
freq: int,
pulseWidth: int,
isContinuous: int
} 

the state of the ground and reference nodes are trickled back through the SOH.


#### 

* there is no state stored on the box
* multiple stim packets can be appended to the stim_req_json, these multiple packets constitute the full state configuration of the box, minus the ground configuration packets.
* ground and reference can be viewed in the state of health, but the 
* stim_req_packet contains the full anode.



#### more notes:

* the box won't trickle back the json to the logger that I need unless stim sequences are actually sent. 
* everything that I will need from a graphing perspective will come through my connection to the logger (except for DJ stuff, which is it's own webserver)

 

In [ ]:
# minimal definition of State of Health packet
soh_msg = {}
soh_msg['msg'] = {}
soh_msg['msg']['stimResCaud'] = 10        # future?
soh_msg['msg']['stimResRost'] = 10        # future?
soh_msg['msg']['xipStimResCaud'] = 10     # future?
soh_msg['msg']['xipStimResRost'] = 10     # future?
soh_msg['msg']['gnd'] = [1,2]             # define which electrodes are ground electrodes
soh_msg['msg']['ref'] = [3,4]             # define which electrodes are reference electrodes
soh_msg['msg']['aspectRatio'] = 3         # define the stimulation aspect ratio


# minimal definition of a stim event packets
stim_event_msg  = {}
stim_event_msg['msg'] = {}
stim_event_msg['msg']['elecCath'] = [31,133]
stim_event_msg['msg']['elecAno'] =  [15,130]
st


{
elecCath: int OR list of ints,
elecAno: int OR list of ints,
amp: int,
freq: int,
pulseWidth: int,
isContinuous: int
} 


In [ ]:
##!/usr/bin/env python
import asyncio
import websockets
import json

# define some global parameters.end
msgFreq = 5 #[Hz] frequency that our message is sent

#connection event handler
async def handler(websocket):
    #ID_REQ handshake
    await websocket.send("ID_REQ")     # ask the client for their ID status
    while True:
        message = await websocket.recv()
        if message == "1":
            break
    
    #subscribe client to the message stream at regular intervals
    while True: 
        await asyncio.sleep(1/msgFreq)
        await websocket.send(json.dumps(soh_msg)) 


#serve
async def main():
    async with websockets.serve(handler, "", 7890):
        await asyncio.Future()  # run forever

#entrypoint
#await main()
loop = asyncio.get_event_loop()
serverTask = loop.create_task(main())
serverTask

### Cancel the server

* run the cell below to cancel the server task

In [ ]:
serverTask.cancel()

### Manual Modification of logging packets for interactive testing

* modify the message fields here to change the frontend behavior.

In [1]:
import websockets
import asyncio
import json

class Storage:
    def __init__(self):
        self.msg = ""

storage = Storage()

async def listen():
    ip_addr = "192.168.42.1"
    port = "7890"
    url = "ws://" + ip_addr + ":" + port

    async with websockets.connect(url) as ws:
        # connect and provide ID - determine the class of listener.
        while True:                                     # ID check loop
            msg = await ws.recv()                       # Non-blocking, waits for a new message to arrive from the server
            if msg == "ID_REQ":                         # If the message is an ID Request
                my_client_id = 1                        # Fetch this client's ID, in this case it's hardcoded
                await ws.send(str(my_client_id))        # Non-blocking, sends the client ID
                print("connected Successfully as logger")
                break                                   # out of ID check loop
        
        #read a single message then break
        while True:
            msg = await ws.recv()                       # Non-blocking, waits for a new message to arrive from the server
            storage.msg = msg
            break
        #await ws.close()


#entrypoint
#await main()
loop = asyncio.get_event_loop()
serverTask = loop.create_task(listen())
serverTask

<Task pending name='Task-4' coro=<listen() running at C:\Users\adawsone\AppData\Local\Temp\ipykernel_14924\3521875238.py:11>>

connected Successfully as logger


In [ ]:
serverTask.cancel()

In [15]:
storage.msg

'{\n    "client_ip": "192.168.42.1",\n    "client_id": 0,\n    "msg_type": "soh_json",\n    "msg": "{\\n    \\"curTime_ns\\": 9531884090133,\\n    \\"unixTime\\": 1686405411.3700488,\\n    \\"nipTime_30k\\": 278285310,\\n    \\"connectedLoggers\\": [\\n        \\"192.168.42.156\\"\\n    ],\\n    \\"connectedReadOnly\\": [\\n        \\"192.168.42.156\\"\\n    ],\\n    \\"connectedReadWrite\\": [\\n        \\"192.168.42.156\\",\\n        \\"192.168.42.156\\"\\n    ],\\n    \\"stimResCaud\\": 10,\\n    \\"stimResRost\\": 10,\\n    \\"xipStimResCaud\\": 0,\\n    \\"xipStimResRost\\": [],\\n    \\"gnd\\": [],\\n    \\"ref\\": [],\\n    \\"aspectRatio\\": 3\\n}"\n}'

In [12]:
msgDict = json.loads(storage.msg)
msgDict['msg'] = json.loads(msgDict['msg'])

In [13]:
json.dumps(msgDict)

'{"client_ip": "192.168.42.1", "client_id": 0, "msg_type": "soh_json", "msg": {"curTime_ns": 9531884090133, "unixTime": 1686405411.3700488, "nipTime_30k": 278285310, "connectedLoggers": ["192.168.42.156"], "connectedReadOnly": ["192.168.42.156"], "connectedReadWrite": ["192.168.42.156", "192.168.42.156"], "stimResCaud": 10, "stimResRost": 10, "xipStimResCaud": 0, "xipStimResRost": [], "gnd": [], "ref": [], "aspectRatio": 3}}'

In [9]:
print(msgDict)

{'client_ip': '192.168.42.1', 'client_id': 0, 'msg_type': 'soh_json', 'msg': '{\n    "curTime_ns": 9531884090133,\n    "unixTime": 1686405411.3700488,\n    "nipTime_30k": 278285310,\n    "connectedLoggers": [\n        "192.168.42.156"\n    ],\n    "connectedReadOnly": [\n        "192.168.42.156"\n    ],\n    "connectedReadWrite": [\n        "192.168.42.156",\n        "192.168.42.156"\n    ],\n    "stimResCaud": 10,\n    "stimResRost": 10,\n    "xipStimResCaud": 0,\n    "xipStimResRost": [],\n    "gnd": [],\n    "ref": [],\n    "aspectRatio": 3\n}'}
